In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "C:\\Users\\i076453\\Downloads\\tech\\ml\\cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """ read data from data file """
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)     # 不做filter, 将所有的数据都加载
            all_labels.append(labels)
        self._data = np.vstack(all_data)  # 纵向合并成
        self._data = self._data / 127.5 - 1 # 进行归一化 
            
        self._labels = np.hstack(all_labels) # 横向合并
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
        
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)] 
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
def inception_block(x, ouput_channel_for_each_path, name):
    """inception block implementation"""
    """
    Args:
    - x
    - ouput_channel_for_each_path eg: [2,3,4,5]
    - name
    """
    
    with tf.variable_scope(name):
        conv1_1 = tf.keras.layers.Conv2D(
                        ouput_channel_for_each_path[0],     # output channel number
                        (1,1),  # kernel size
                        strides = (1,1),# 步长
                        padding = 'same',  # 表示输出输入图像大小一致
                        activation=tf.nn.relu, # 激活函数
                        name = 'conv1_1' # 名字
                        )(x)
        
        conv3_3 = tf.keras.layers.Conv2D(
                        ouput_channel_for_each_path[1],     # output channel number
                        (3,3),  # kernel size
                        strides = (1,1),# 步长
                        padding = 'same',  # 表示输出输入图像大小一致
                        activation=tf.nn.relu, # 激活函数
                        name = 'conv3_3' # 名字
                        )(x)
        
        conv5_5 = tf.keras.layers.Conv2D(
                        ouput_channel_for_each_path[1],     # output channel number
                        (5,5),  # kernel size
                        strides = (1,1),# 步长
                        padding = 'same',  # 表示输出输入图像大小一致
                        activation=tf.nn.relu, # 激活函数
                        name = 'conv5_5' # 名字
                        )(x)
        
        max_pooling = tf.keras.layers.MaxPool2D(
                                              (2,2), # kernel size
                                              (2,2), # stride 步长 
                                              name = 'max_pooling')(x)
        
    max_pooling_shape = max_pooling.get_shape().as_list()[1:]
    input_shape = x.get_shape().as_list()[1:]
    width_padding = (input_shape[0] - max_pooling_shape[0]) // 2   # 问题： 为什么这样算？
    height_padding = (input_shape[1] - max_pooling_shape[1]) // 2
    
    # 只在宽，高上做padding, 不在上下做padding
    pad_pooling = tf.pad(max_pooling, 
                        [[0,0],
                        [width_padding, width_padding],
                        [height_padding, height_padding],
                        [0,0]])
    
    # 拼接这些层，他们都在同一层级
    concat_layer = tf.concat(
        [conv1_1, conv3_3, conv5_5, pad_pooling],
        axis=3 # 需要在第四个通道上拼接，第四个通道的index是3
    )
    return concat_layer
    

x = tf.placeholder(tf.float32, [None, 3072]) # data 的placeholder, None表示不确定的
# y的shape是 [0.6,5,3]
y = tf.placeholder(tf.int64, [None]) # labels的placeholder

# 转换成多通道
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 交换通道
# 32 x 32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1]) # [-1, 32, 32, 3]


# conv1 : 神经元图，feature_map, 输出图像
conv1 = tf.keras.layers.Conv2D(
                        32,     # output channel number
                        (3,3),  # kernel size
                        padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                        activation=tf.nn.relu, # 激活函数
                        name = 'conv1' # 名字
                        )(x_image)

# 构建池化层
# 16 x 16 就是将原来的图片大小降为原来的1/2
pooling1 = tf.keras.layers.MaxPool2D(
                                  (2,2), # kernel size
                                  (2,2), # stride 步长 
                                  name = 'pooling1')(conv1)

# 添加两个inception层, 每个inception层都有4个卷积形成一组
inception_2a = inception_block(pooling1, [16,16,16], name = "inception_2a")
inception_2b = inception_block(inception_2a, [16,16,16], name = "inception_2b")

# 构建池化层
# 16 x 16 就是将原来的图片大小降为原来的1/2
pooling2 = tf.keras.layers.MaxPool2D(
                                  (2,2), # kernel size
                                  (2,2), # stride 步长 
                                  name = 'pooling2')(inception_2b)


# 添加两个inception层
inception_3a = inception_block(pooling2, [16,16,16], name = "inception_3a")
inception_3b = inception_block(inception_3a, [16,16,16], name = "inception_3b")

# 构建池化层
# 16 x 16 就是将原来的图片大小降为原来的1/2
pooling3 = tf.keras.layers.MaxPool2D(
                                  (2,2), # kernel size
                                  (2,2), # stride 步长 
                                  name = 'pooling3')(inception_3b)

# 展平图片 [None, 4 x 4 x 32]
flatten = tf.keras.layers.Flatten()(pooling3)
    
# 构建全连接层
y_ = tf.keras.layers.Dense(10)(flatten)

# 交叉熵
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 它做了下面三件事
# y_ -> softmax
# y -> one hot
# loss = ylogy_

# bool 
predict = tf.argmax(y_, 1) # 在第二个维度求最大值

# [1,0,0,1,0,0,1,1...]
correct_predict = tf.equal(predict, y)

# 准确率
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))

# 有了loss目标函数和准确率后，再去做一个
# AdamOptimizer 是一个剔除向量的变种，1e-3就是10的-3次方0.001 
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
#  run 10000, acc: 0.73550
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size) 
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 500, loss: 1.22817, acc: 0.35000
[Train] Step: 1000, loss: 0.89484, acc: 0.70000
[Train] Step: 1500, loss: 0.66580, acc: 0.75000
[Train] Step: 2000, loss: 0.64331, acc: 0.70000
[Train] Step: 2500, loss: 1.21480, acc: 0.60000
[Train] Step: 3000, loss: 0.80636, acc: 0.65000
[Train] Step: 3500, loss: 0.97204, acc: 0.50000
[Train] Step: 4000, loss: 0.91716, acc: 0.75000
[Train] Step: 4500, loss: 1.00470, acc: 0.65000
[Train] Step: 5000, loss: 1.06456, acc: 0.55000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.71600
[Train] Step: 5500, loss: 0.61810, acc: 0.75000
[Train] Step: 6000, loss: 0.49863, acc: 0.85000
[Train] Step: 6500, loss: 0.44663, acc: 0.90000
[Train] Step: 7000, loss: 0.65775, acc: 0.70000
[Train] Step: 7500, loss: 0.97029, acc: 0.55000
[Train] Step: 8000, loss: 0.94312, acc: 0.70000
[Train] Step: 8500, loss: 0.42665, acc: 0.80000
[Train] Step: 9000, loss: 0.76164, acc: 0.80000
[Train] Step: 9500, loss: 0.68030, acc: 0.75000
[Train] Step: 10000, loss: 0.28058